In [4]:
import os
import sys
import matplotlib
import matplotlib.pyplot as plt
import dask.array as da
import dask.dataframe as dd
import xarray as xr
from xnemogcm import open_domain_cfg, get_metrics
import xgcm
import cartopy.crs as ccrs
import cmocean
import numpy as np
from scipy.stats import linregress
import datetime
import pandas as pd
import geopy
# Add SouthernDemons library to PATH
sys.path.append(os.path.abspath("../lib/"))
from teos_ten import teos_sigma0
import datesandtime

In [5]:
# Subdomain information (As inputted into TRACMASS, note non-pythonic indexing)
imindom = 1
imaxdom = 1440
jmindom = 1
jmaxdom = 400
kmindom = 1
kmaxdom = 75

# Location of the TRACMASS run


# Location of masks and grid information for the model
grid_path = os.path.abspath("/gws/nopw/j04/bas_pog/astyles/ORCA025_fwd/topo" )
grid_files = ['mask.nc','mesh_hgr.nc','mesh_zgr.nc']

In [6]:
ds_domain = open_domain_cfg( datadir=grid_path, files = grid_files )

/home/users/zhenya/.conda/envs/ventilation/lib/python3.13/site-packages/dask/dataframe/core.py:375: UserWarning: Insufficient elements for `head`. 2 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(
/home/users/zhenya/.conda/envs/ventilation/lib/python3.13/site-packages/dask/dataframe/core.py:375: UserWarning: Insufficient elements for `head`. 3 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(
/home/users/zhenya/.conda/envs/ventilation/lib/python3.13/site-packages/dask/dataframe/core.py:375: UserWarning: Insufficient elements for `head`. 4 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(
/home/users/zhenya/.conda/envs/ventilation/lib/python3.13/site-packages/dask/dataframe/core.py:375: UserWarning: Insufficient elements for `head`. 4 elements requested, only 1 elements available. Try passing larger `npartitions` 

In [84]:
ds_domain

<xarray.Dataset> Size: 6GB
Dimensions:       (y_c: 1021, x_c: 1442, x_f: 1442, y_f: 1021, z_c: 75, z_f: 75)
Coordinates: (12/18)
    glamt         (y_c, x_c) float32 6MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    glamu         (y_c, x_f) float32 6MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    glamv         (y_f, x_c) float32 6MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    glamf         (y_f, x_f) float32 6MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    gphit         (y_c, x_c) float32 6MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    gphiu         (y_c, x_f) float32 6MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    ...            ...
  * x_f           (x_f) float64 12kB 0.5 1.5 2.5 ... 1.44e+03 1.44e+03 1.442e+03
  * y_f           (y_f) float64 8kB 0.5 1.5 2.5 ... 1.018e+03 1.02e+03 1.02e+03
  * z_f           (z_f) float64 600B -0.5 0.5 1.5 2.5 ... 70.5 71.5 72.5 73.5
  * x_c           (x_c) int64 12kB 0 1 2 3 4 5 ... 1436 1437 1438 1439 1440 1441
  * y_c           (y_c) int64 8kB 0 1 2 3 4 5 ... 1015 1016 1017 1018 1019 1020
  * z_c           (z_c) int64 600B 0 1 2 3 4 5 6 7 8 ... 67 68 69 70 71 72 73 74
Data variables: (12/29)
    time_counter  float64 8B dask.array<chunksize=(), meta=np.ndarray>
    e1t           (y_c, x_c) float64 12MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    e1u           (y_c, x_f) float64 12MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    e1v           (y_f, x_c) float64 12MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    e1f           (y_f, x_f) float64 12MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    e2t           (y_c, x_c) float64 12MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    ...            ...
    vmask         (z_c, y_f, x_c) int8 110MB dask.array<chunksize=(5, 128, 91), meta=np.ndarray>
    fmask         (z_c, y_f, x_f) int8 110MB dask.array<chunksize=(5, 128, 91), meta=np.ndarray>
    tmaskutil     (y_c, x_c) int8 1MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    umaskutil     (y_c, x_f) int8 1MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    vmaskutil     (y_f, x_c) int8 1MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    fmaskutil     (y_f, x_f) int8 1MB dask.array<chunksize=(128, 91), meta=np.ndarray>
Attributes:
    DOMAIN_size_global:  [1442 1021]

In [ ]:
#logic
#create array of x dist in meters,  shape = (n) x (n) (as it has a periodic b.c.)
#create array of y dist in meters, shape = (n) x (n)
#area = multiply both

In [ ]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.geodesic(coords_1, coords_2).km)

from geopy import distance 
distance_2d= distance.distance(pt1, pt2).m
print(distance_2d)


In [44]:
import numpy as np
# points with latitude,longitude units in degrees ,altitude unit in meters
df = dd.DataFrame.from_dict({"lon1":[1,2], "lat1":[3,4], 'lon2':[5,6], 'lat2':[7,8]} , npartitions=4)
#df['x_dist'] = distance.distance([df['lon1'],df['lat1']], [df['lon2'],df['lat2']]).m
# 2D geodesic distance in meters


In [9]:
pt1[:1]

[35.3524]

In [40]:
df.compute()

,lon1,lat1,lon2,lat2
0,1,3,5,7
1,2,4,6,8


In [45]:
##### start again

In [66]:
trunc = ds_domain.head(5)

In [71]:
lat = ds_domain.gphit
lon = ds_domain.glamt

In [75]:
lon[0].compute()

<xarray.DataArray 'glamt' (x_c: 1442)> Size: 6kB
array([72.75, 73.  , 73.25, ..., 72.5 , 72.75, 73.  ],
      shape=(1442,), dtype=float32)
Coordinates:
    glamt    (x_c) float32 6kB 72.75 73.0 73.25 73.5 ... 72.25 72.5 72.75 73.0
    gphit    (x_c) float32 6kB -77.01 -77.01 -77.01 ... -77.01 -77.01 -77.01
  * x_c      (x_c) int64 12kB 0 1 2 3 4 5 6 ... 1436 1437 1438 1439 1440 1441
    y_c      int64 8B 0
Attributes:
    standard_name:  longitude
    units:          degrees_east

In [79]:
lon[-10].compute()

<xarray.DataArray 'glamt' (x_c: 1442)> Size: 6kB
array([72.968834, 73.      , 73.031166, ..., 72.93643 , 72.968834,
       73.      ], shape=(1442,), dtype=float32)
Coordinates:
    glamt    (x_c) float32 6kB 72.97 73.0 73.03 73.06 ... 72.91 72.94 72.97 73.0
    gphit    (x_c) float32 6kB 49.97 49.96 49.97 50.01 ... 50.01 49.97 49.96
  * x_c      (x_c) int64 12kB 0 1 2 3 4 5 6 ... 1436 1437 1438 1439 1440 1441
    y_c      int64 8B 1011
Attributes:
    standard_name:  longitude
    units:          degrees_east